<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/datasets/try_argilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We follow 

- https://docs.argilla.io/en/latest/tutorials/notebooks/labelling-textclassification-sentence-transformers-semantic.html
- https://github.com/argilla-io/argilla/blob/develop/scripts/load_data.py

but apply to our own dataset and argilla instance.

In [1]:
%pip install argilla datasets==2.8.0 sentence-transformers==2.2.2  -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 23.0 MB/s eta 0:00

In [2]:
import argilla as rg

In [3]:
rg.init(
    api_url="https://utensil-argilla.hf.space/",
    api_key="admin.apikey"
)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:152: UserWarning: You're connecting to Argilla Server 1.9.0-dev0 using a different client version (1.8.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [4]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

In [5]:
# Define fast version of sentence transformers
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [19]:
from datasets import load_dataset

dataset = load_dataset("QingyiSi/Alpaca-CoT", data_files ="Chain-of-Thought/formatted_cot_data/gsm8k_train.json", split="train")

In [20]:
dataset.to_pandas().head()

,instruction,input,output
0,Natalia sold clips to 48 of her friends in Apr...,,Natalia sold 48 / 2 = 24 clips in May. Natalia...
1,Lizzy: Weng earns $12 an hour for babysitting....,,Weng earns 12 / 60 = $0.2 per minute. Working ...
2,Give the step-by-step reasoning process and th...,,"In the beginning, Betty has only 100 / 2 = $50..."
3,Question: Julie is reading a 120-page book. Ye...,,Maila read 12 x 2 = 24 pages today. So she was...
4,James writes a 3-page letter to 2 different fr...,,He writes each friend 3 * 2 = 6 pages a week. ...


In [21]:
dataset.data.shape

(7473, 3)

In [22]:
# Encode text field using batched computation
vd = dataset.map(
    lambda batch: {"vectors": encoder.encode(batch["instruction"]
        ),
    },
    batched=True,
    batch_size=32
)

  0%|          | 0/234 [00:00<?, ?ba/s]

In [23]:
vd.data.shape

(7473, 4)

In [24]:
vd.to_pandas().head()

,instruction,input,output,vectors
0,Natalia sold clips to 48 of her friends in Apr...,,Natalia sold 48 / 2 = 24 clips in May. Natalia...,"[0.0087681115, -0.048527658, 0.081025615, -0.0..."
1,Lizzy: Weng earns $12 an hour for babysitting....,,Weng earns 12 / 60 = $0.2 per minute. Working ...,"[-0.0025617, 0.084493876, 0.076010056, 0.04024..."
2,Give the step-by-step reasoning process and th...,,"In the beginning, Betty has only 100 / 2 = $50...","[0.045931183, 0.1536832, -0.0038327056, -0.063..."
3,Question: Julie is reading a 120-page book. Ye...,,Maila read 12 x 2 = 24 pages today. So she was...,"[0.061020136, 0.06757341, -0.0059414604, -0.02..."
4,James writes a 3-page letter to 2 different fr...,,He writes each friend 3 * 2 = 6 pages a week. ...,"[0.0141281085, 0.04150687, 0.07270662, -0.0285..."


In [25]:
# Turn vectors into a dictionary
vdm = vd.map(
    lambda r: 
     {
        "text": f'USER: {r["instruction"]}\nASSISTANT: {r["output"]}',
        "vectors": {"mini-lm-sentence-transformers": r["vectors"]}
     }
    # remove_columns=vd.column_names
)

  0%|          | 0/7473 [00:00<?, ?ex/s]

In [27]:
dataset_rg = rg.read_datasets(vdm, task="Text2Text")

In [28]:
# Log the dataset
rg.log(
    dataset_rg,
    name="gsm-8k-test",
    tags={"description": "Testing gsm-8k"},
)

Output()

7473 records logged to ]8;id=624165;https://utensil-argilla.hf.space/datasets/admin/gsm-8k-test\https://utensil-argilla.hf.space/datasets/admin/gsm-8k-test]8;;\

BulkResponse(dataset='gsm-8k-test', processed=7473, failed=0)

In [47]:
v = vdm.to_pandas().head(1)['vectors'][0]['mini-lm-sentence-transformers']
v

array([ 8.76811147e-03, -4.85276580e-02,  8.10256153e-02, -2.26558466e-03,
       -4.04507369e-02,  4.98062260e-02,  8.03768262e-02,  6.82035759e-02,
       -7.24245096e-03,  6.23441720e-03,  6.15905784e-02,  3.00756097e-02,
        3.23917135e-03, -3.56388204e-02, -5.26666548e-03,  9.66139659e-02,
       -6.49311393e-02,  3.97711433e-02, -7.81763718e-02, -1.70678906e-02,
       -2.55708005e-02, -1.84912294e-01, -7.04867253e-03, -2.80134450e-03,
        1.21055856e-01,  3.34485136e-02, -6.51538447e-02, -4.38730493e-02,
       -6.70669973e-03, -1.57886930e-02,  1.29787158e-02,  9.46261641e-03,
        1.82289872e-02,  5.01755103e-02, -1.56352092e-02, -6.36283755e-02,
       -1.32085206e-02, -5.46702407e-02,  3.70722637e-03,  3.52124423e-02,
       -1.05476342e-02, -9.15647224e-02, -7.56042749e-02,  1.65473646e-03,
       -2.10455079e-02,  1.05940588e-02, -6.23101965e-02,  1.30693406e-01,
        1.91438198e-02,  2.97926161e-02, -5.40030375e-02,  5.85508794e-02,
       -1.57515168e-01, -

In [48]:
v.shape

(384,)